In [1]:
#импортируем используемые библиотеки

import pandas as pd
import numpy as np

In [2]:
#читаем файл с которым будем работать

data = pd.read_csv('main_task_new.csv')

In [26]:
#смотри на dataframe

data.head()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


In [3]:
#делаем копию, чтобы откатиться, если что-то пойдет не так

df = data.copy()

In [4]:
#убираю, то с чем не буду работать

df = df.drop(columns = ['Restaurant_id', 'Reviews', 'URL_TA', 'ID_TA'])

In [5]:
#начинаю заполнять пропуски

top_price_range = df['Price Range'].value_counts().idxmax()
df['Price Range'] = df['Price Range'].fillna(top_price_range)

In [6]:
#готовимся создать категориальные переменные

df['Price Range'].unique()

array(['$$ - $$$', '$$$$', '$'], dtype=object)

In [7]:
price_range = {'$':0,
               '$$ - $$$':1,
               '$$$$':2,
               }

In [9]:
#создаем категориальные переменные

df['Price Range'] = df['Price Range'].replace(to_replace = price_range)

In [10]:
#заполняем пропуски непрерывных переменных

mean_number_of_reviews = df['Number of Reviews'].mean()
df['Number of Reviews'] = df['Number of Reviews'].fillna(mean_number_of_reviews)

In [11]:
#удаляю, то что решил не выдумывать

df.dropna(inplace = True)

In [12]:
#из оставшегося, создаем новые переменные

cuisine_list = df['Cuisine Style'].copy()
cuisine_list.replace(regex=True, inplace=True, to_replace=r'[\[\]\'\'\s]', value=r'')
cuisine_list = cuisine_list.str[:].str.split(',')

In [13]:
cuisine_count = []
for i in cuisine_list:
    cuisine_count.append(len(i))

In [14]:
df['cuisine_count'] = cuisine_count

In [15]:
#после создания новых переменных, удаляем, старые

df = df.drop(['Cuisine Style'], axis = 1)

In [16]:
#создаем фиктивные переменные

df1 = pd.get_dummies(df['City'])

In [17]:
#создаем финальный датафрейм на котором будет проходить обучение

final_data = pd.concat([df, df1], axis = 1)

In [18]:
#на всякий случай делаем копию, для отката

fd = final_data.copy()

In [20]:
X = fd.drop(['City', 'Rating'], axis = 1)
y = fd['Rating']

In [21]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [23]:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [24]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [25]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.219890625
